In [ ]:
import pandas as pd
import numpy as np

file = "../data/facebook.csv"

df = pd.read_csv(file, low_memory=False, parse_dates=[0])

df.head()

In [ ]:
%cd ..

In [ ]:
from src.clean_data import clean_df

In [ ]:
new_df = clean_df(df)
new_df.head()

In [ ]:
impressions = []
views = []
clicks = []
others = []

for col in new_df.columns:
    if "Impressions" in col:
        impressions.append(col)
    elif "Clicks" in col:
        clicks.append(col)
    elif "Views" in col:
        views.append(col)
    else:
        others.append(col)